In [1]:
import mne
import os
from ieeg.viz.mri import plot_subj, plot_subj_sig_and_nonsig, _plot_electrodes, subject_to_info, plot_on_average
from ieeg.io import raw_from_layout, get_data
from misc_functions import load_sig_chans, channel_names_to_indices
import json

### plot single subject's significant electrodes

In [ ]:
subj = "D0100"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'
stim_sig_chans = load_sig_chans(stim_filename)
# resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response_fixationCrossBase_1sec_mirror.json'
# resp_sig_chans = load_sig_chans(stim_filename)


subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj(subj_without_zeroes, picks=stim_sig_chans, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 
# plot_subj(subj_without_zeroes, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', color=(1, 1, 0))

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_sigStim.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')


In [ ]:
subj = "D0063"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'
congruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_congruent.json'
incongruent_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_incongruent.json'
repeat_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_repeat.json'
switch_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_switch.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)
congruent_sig_chans = load_sig_chans(congruent_filename)
incongruent_sig_chans = load_sig_chans(incongruent_filename)
repeat_sig_chans = load_sig_chans(repeat_filename)
switch_sig_chans = load_sig_chans(switch_filename)


subj_without_zeroes = "D" + subj[1:].lstrip('0')
plot_subj(subj_without_zeroes, picks=stim_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=resp_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Response', background='black', color=(1, 1, 0))
plot_subj(subj_without_zeroes, picks=congruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Congruent', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=incongruent_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Incongruent', background='black', color=(1, 1, 0))
plot_subj(subj_without_zeroes, picks=repeat_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Repeat', background='black', color=(1, 1, 0)) 
plot_subj(subj_without_zeroes, picks=switch_sig_chans, labels_every=8, hemi='both', title=f'{subj}_Switch', background='black', color=(1, 1, 0))


### plot all significant electrodes on average brain

In [2]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    # skip d0107 for now cuz i haven't preprocessed them yet 3/25
    if subj == "D0107":
        continue
    
    # resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_response_experimentStartBase1secTo101Sec.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'

    # resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    # combined_chans = resp_sig_chans + stim_sig_chans
    combined_chans = stim_sig_chans # just plot stimulus for now, not response
    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

picks = [f'D{k[1:].zfill(4)}-{v}' for k in allSigElectrodes.keys() for v in allSigElectrodes[k]]
# sinfos = [subject_to_info(f'D{int(subj[1:])}') for subj in subjects]

plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0), size=0.45)
plot_on_average(allElectrodes, hemi='both', color=(1,1,0))

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103
Using pyvistaqt 3d backend.



C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

### plot all significant electrodes, then plot all electrodes

In [3]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")

subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]

allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

for subj in subjects:
    # if subj == "D0077":
    #     continue
    
    # resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
    stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus_fixationCrossBase_1sec_mirror.json'

    # resp_sig_chans = load_sig_chans(resp_filename)
    stim_sig_chans = load_sig_chans(stim_filename)

    # Combine the significant electrodes for this subject
    # combined_chans = resp_sig_chans + stim_sig_chans
    combined_chans = stim_sig_chans # delete this line once we have responses for everyone
    # Add them to the dictionary, tagged with the subject without zeroes

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    allSigElectrodes[subj_without_zeroes] = combined_chans

    # Get all electrodes for this subject and add them to the allElectrodes dict
    info = subject_to_info(subj_without_zeroes)
    allSubjElectrodes = mne.pick_channels(info.ch_names, [])
    allElectrodes[subj_without_zeroes] = allSubjElectrodes

# apparently this doesn't work because the inputs are dicts and plot on average doesn't like dict inputs.
# use Aaron's code below that converts from these dicts to a list of electrode indices 3/25. 
# fig = plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0), size=0.45)
# plot_on_average(allElectrodes, hemi='both', color=(1,1,1), fig=fig)

{'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48}
KeysView({'derivatives/clean': BIDS Layout: ...alLocal\BIDS\derivatives\clean | Subjects: 12 | Sessions: 0 | Runs: 48})
Loaded significant channels for subject D0057
Loaded significant channels for subject D0059


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


aaron's way of plotting

In [4]:
allSigElectrodes
subjects = []
length = 0
elecs = []
for subject, electrodes in allSigElectrodes.items():
    info = subject_to_info(subject)
    for electrode in electrodes:
        elecs.append(length + info['ch_names'].index(electrode))
    length += len(info['ch_names'])
    subjects.append(subject)

fig = plot_on_average(subjects, picks=elecs, rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4)
plot_on_average(subjects, rm_wm=False, hemi='both', color=(1, 1, 1), fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [10]:
info = subject_to_info('D63')
info['ch_names']

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


['LOF16',
 'LOF15',
 'LOF14',
 'LOF13',
 'LOF12',
 'LOF11',
 'LOF10',
 'LOF9',
 'LOF8',
 'LOF7',
 'LOF6',
 'LOF5',
 'LOF4',
 'LOF3',
 'LOF2',
 'LOF1',
 'LASF16',
 'LASF15',
 'LASF14',
 'LASF13',
 'LASF12',
 'LASF11',
 'LASF10',
 'LASF9',
 'LASF8',
 'LASF7',
 'LASF6',
 'LASF5',
 'LASF4',
 'LASF3',
 'LASF2',
 'LASF1',
 'LMSF12',
 'LMSF11',
 'LMSF10',
 'LMSF9',
 'LMSF8',
 'LMSF7',
 'LMSF6',
 'LMSF5',
 'LMSF4',
 'LMSF3',
 'LMSF2',
 'LMSF1',
 'LPSF14',
 'LPSF13',
 'LPSF12',
 'LPSF11',
 'LPSF10',
 'LPSF9',
 'LPSF8',
 'LPSF7',
 'LPSF6',
 'LPSF5',
 'LPSF4',
 'LPSF3',
 'LPSF2',
 'LPSF1',
 'LMMT14',
 'LMMT13',
 'LMMT12',
 'LMMT11',
 'LMMT10',
 'LMMT9',
 'LMMT8',
 'LMMT7',
 'LMMT6',
 'LMMT5',
 'LMMT4',
 'LMMT3',
 'LMMT2',
 'LMMT1',
 'RPIT14',
 'RPIT13',
 'RPIT12',
 'RPIT11',
 'RPIT10',
 'RPIT9',
 'RPIT8',
 'RPIT7',
 'RPIT6',
 'RPIT5',
 'RPIT4',
 'RPIT3',
 'RPIT2',
 'RPIT1',
 'RMIT14',
 'RMIT13',
 'RMIT12',
 'RMIT11',
 'RMIT10',
 'RMIT9',
 'RMIT8',
 'RMIT7',
 'RMIT6',
 'RMIT5',
 'RMIT4',
 'RMIT3',

some testing to see whats different between d63 and d103 in ch info and the channels tsv. Delete once figure out whats different 3/26.

In [15]:
import csv

# Replace 'your_tsv_file.tsv' with the path to your TSV file
tsv_file_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\sub-D0103\ieeg\sub-D0103_task-GlobalLocal_channels.tsv'

# Initialize an empty list to hold the channel names
channel_names = []

# Open the TSV file for reading
with open(tsv_file_path, mode='r', newline='') as file:
    # Create a CSV reader object specifying the delimiter as a tab
    reader = csv.reader(file, delimiter='\t')
    
    # Skip the header row (if your file has a header)
    next(reader)
    
    # Iterate over each row in the TSV file
    for row in reader:
        # The first item in each row is the channel name
        channel_name = row[0]
        channel_names.append(channel_name)

# Now channel_names list contains all the channel names from the TSV file
print(channel_names)

set_tsv_d63 = set(channel_names)
set_chinfo_d63 = set(subject_to_info('D103')['ch_names'])
differences = set_chinfo_d63 - set_tsv_d63
differences



['LTAS1', 'LTAS2', 'LTAS3', 'LTAS4', 'LTAS5', 'LTAS6', 'LTAS7', 'LTAS8', 'LTMS1', 'LTMS2', 'LTMS3', 'LTMS4', 'LTMS5', 'LTMS6', 'LTMS7', 'LTMS8', 'LTMS9', 'LTMS10', 'LTPS1', 'LTPS2', 'LTPS3', 'LTPS4', 'LTPS5', 'LTPS6', 'LTPS7', 'LTPS13', 'LTPS14', 'LTPS15', 'LTPS16', 'LTPS17', 'LTPS18', 'LTAI1', 'LTAI2', 'LTAI3', 'LTAI4', 'LTAI5', 'LTAI6', 'LTAI7', 'LTAI8', 'LTAI9', 'LTAI10', 'LTAI11', 'LTAI12', 'LTAI13', 'LTAI14', 'LTAI15', 'LFAM1', 'LFAM2', 'LFAM3', 'LFAM4', 'LFAM5', 'LFAM6', 'LFAM7', 'LFAM8', 'LFAM9', 'LFAM10', 'LFAM11', 'LFAM12', 'LFAM13', 'LFAM14', 'LFAM15', 'LAI1', 'LAI2', 'LAI3', 'LAI4', 'LAI5', 'LAI6', 'LAI7', 'LAI8', 'LAI12', 'LAI13', 'LAI14', 'LAI15', 'LAI16', 'LAI17', 'LAI18', 'LPI1', 'LPI2', 'LPI3', 'LPI4', 'LPI5', 'LPI6', 'LPI7', 'LPI8', 'LPI15', 'LPI16', 'LPI17', 'LPI18', 'LTP1', 'LTP2', 'LTP3', 'LTP4', 'LTP5', 'LTP6', 'LTP7', 'LTP8', 'LTP9', 'LTP10', 'LTP11', 'LTP12', 'LTAM1', 'LTAM2', 'LTAM3', 'LTAM4', 'LTAM5', 'LTAM6', 'LTAM7', 'LTAM8', 'LTAM9', 'LTAM10', 'LTAM11', 'LTA

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


{'LAI10',
 'LAI11',
 'LAI9',
 'LFAI10',
 'LFO13',
 'LPI10',
 'LPI11',
 'LPI12',
 'LPI13',
 'LPI14',
 'LPI9',
 'LTLI14',
 'LTLI15',
 'LTMI15',
 'LTMM1',
 'LTPM15',
 'LTPM5',
 'LTPS10',
 'LTPS11',
 'LTPS12',
 'LTPS8',
 'LTPS9'}

In [8]:
# Correcting the snippet to handle the dict properly and calculate total number of values
total_values = sum(len(values) for values in allElectrodes.values())
print(total_values)

# Print the total number of values for each key in the allElectrodes dictionary
total_values_each_key = {key: len(values) for key, values in allElectrodes.items()}
print(total_values_each_key)

2387
{'D57': 178, 'D59': 184, 'D63': 246, 'D65': 220, 'D69': 132, 'D71': 162, 'D77': 157, 'D90': 164, 'D94': 240, 'D100': 230, 'D102': 227, 'D103': 247}


plot sig for congruency at any time window and also sig for interaction between congruency and congruency proportion at any time window 3/25

In [11]:
LAB_root = None
channels = None
full_trial_base = False
task = 'GlobalLocal'
if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

congruency_congruencyProportion_structure_file_path = os.path.join(save_dir, 'congruency_congruencyProportion_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(congruency_congruencyProportion_structure_file_path, 'r') as file:
    congruency_congruencyProportion_structure = json.load(file)

switchType_switchProportion_structure_file_path = os.path.join(save_dir, 'switchType_switchProportion_ANOVAwithinElectrodes_significantEffectsStructure_wholeBrainAnalysis.txt')
with open(switchType_switchProportion_structure_file_path, 'r') as file:
    switchType_switchProportion_structure = json.load(file)

In [3]:
# congruency_elecs = []
# switchType_elecs = []
# congruency_congruencyProportion_elecs = []
# switchType_switchProportion_elecs = []
# subjects = []
# added_elecs_congruency = set()  # To keep track of added electrodes for congruency
# added_elecs_congruency_congruencyProportion = set()  # To keep track of added electrodes for congruency_congruencyProportion

# added_elecs_switchType = set()  # To keep track of added electrodes for switch type
# added_elecs_switchType_switchProportion = set()  # To keep track of added electrodes for switchType_switchProportion

# length = 0
# for subject, electrodes_dict in congruency_congruencyProportion_structure.items():
#     subject_without_zeroes = "D" + subject[1:].lstrip('0')
#     info = subject_to_info(subject_without_zeroes)
#     # Ensure length is updated correctly, if intended to track cumulative channel count, update outside the electrode loop
#     current_subject_length = len(info['ch_names'])

#     if subject_without_zeroes not in subjects:
#         subjects.append(subject_without_zeroes)

#     for electrode, time_windows in electrodes_dict.items():
#         for time_window, effects in time_windows.items():
#             for effect_and_p_value in effects:
#                 effect_name = effect_and_p_value[0]
#                 electrode_index = info['ch_names'].index(electrode)
        
#                 if effect_name == 'congruency' and (subject_without_zeroes, electrode) not in added_elecs_congruency:
#                     congruency_elecs.append(length + electrode_index)
#                     added_elecs_congruency.add((subject_without_zeroes, electrode))

#                 if effect_name == 'congruency:congruencyProportion' and (subject_without_zeroes, electrode) not in added_elecs_congruency_congruencyProportion:
#                     congruency_congruencyProportion_elecs.append(length + electrode_index)
#                     added_elecs_congruency_congruencyProportion.add((subject_without_zeroes, electrode))

#     length += current_subject_length  # Update length after processing each subject

# # now do for switch type and switch type: switch proportion too
# length = 0
# for subject, electrodes_dict in switch_switchProportion_structure.items():
#     subject_without_zeroes = "D" + subject[1:].lstrip('0')
#     info = subject_to_info(subject_without_zeroes)
#     # Ensure length is updated correctly, if intended to track cumulative channel count, update outside the electrode loop
#     current_subject_length = len(info['ch_names'])
    
#     if subject_without_zeroes not in subjects:
#         subjects.append(subject_without_zeroes)
        
#     for electrode, time_windows in electrodes_dict.items():
#         for time_window, effects in time_windows.items():
#             for effect_and_p_value in effects:
#                 effect_name = effect_and_p_value[0]
#                 electrode_index = info['ch_names'].index(electrode)
        
#                 if effect_name == 'switchType' and (subject_without_zeroes, electrode) not in added_elecs_switchType:
#                     switchType_elecs.append(length + electrode_index)
#                     added_elecs_switchType.add((subject_without_zeroes, electrode))

#                 if effect_name == 'switchType:switchProportion' and (subject_without_zeroes, electrode) not in added_elecs_switchType_switchProportion:
#                     switchType_switchProportion_elecs.append(length + electrode_index)
#                     added_elecs_switchType_switchProportion.add((subject_without_zeroes, electrode))

#     length += current_subject_length  # Update length after processing each subject

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

In [23]:
congruency_elecs = []
switchType_elecs = []
congruency_congruencyProportion_elecs = []
switchType_switchProportion_elecs = []
subjects = []
added_elecs_congruency = set()  # To keep track of added electrodes for congruency
added_elecs_congruency_congruencyProportion = set()  # To keep track of added electrodes for congruency_congruencyProportion

added_elecs_switchType = set()  # To keep track of added electrodes for switch type
added_elecs_switchType_switchProportion = set()  # To keep track of added electrodes for switchType_switchProportion


def process_effects(electrode, time_windows, subject_without_zeroes, info, length, effect_name, elecs_list, added_elecs_set):
    """A helper function to process electrode effects and update lists and sets accordingly."""
    for time_window, effects in time_windows.items():
        for effect_and_p_value in effects:
            if effect_and_p_value[0] == effect_name and (subject_without_zeroes, electrode) not in added_elecs_set:
                elecs_list.append(length + info['ch_names'].index(electrode))
                added_elecs_set.add((subject_without_zeroes, electrode))
                
# Assuming both structures have the same subjects...
length = 0
for subject in congruency_congruencyProportion_structure:
    subject_without_zeroes = "D" + subject[1:].lstrip('0')
    info = subject_to_info(subject_without_zeroes)

    # Ensure each subject is only added once
    if subject_without_zeroes not in subjects:
        subjects.append(subject_without_zeroes)

    current_subject_length = len(info['ch_names'])

    # Process congruency and congruency:congruencyProportion effects
    electrodes_dict_congruency = congruency_congruencyProportion_structure[subject]
    for electrode, time_windows in electrodes_dict_congruency.items():
        process_effects(electrode, time_windows, subject_without_zeroes, info, length, 'congruency', congruency_elecs, added_elecs_congruency)
        process_effects(electrode, time_windows, subject_without_zeroes, info, length, 'congruency:congruencyProportion', congruency_congruencyProportion_elecs, added_elecs_congruency_congruencyProportion)

    # Process switchType and switchType:switchProportion effects
    electrodes_dict_switch = switchType_switchProportion_structure.get(subject, {})
    for electrode, time_windows in electrodes_dict_switch.items():
        process_effects(electrode, time_windows, subject_without_zeroes, info, length, 'switchType', switchType_elecs, added_elecs_switchType)
        process_effects(electrode, time_windows, subject_without_zeroes, info, length, 'switchType:switchProportion', switchType_switchProportion_elecs, added_elecs_switchType_switchProportion)

    length += current_subject_length  # Update length after processing each subject

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot unique congruency, unique congruency and congruency proportion interaction, and also their overlap

In [27]:
# Convert lists to sets for set operations
congruency_elecs_set = set(congruency_elecs)
congruency_congruencyProportion_elecs_set = set(congruency_congruencyProportion_elecs)

# Find overlapping electrodes (present in both lists)
overlap_congruencyEffectAndLWPC_elecs = congruency_elecs_set.intersection(congruency_congruencyProportion_elecs_set)

# Find unique electrodes in congruency_elecs
unique_congruency_elecs = congruency_elecs_set - congruency_congruencyProportion_elecs_set

# Find unique electrodes in congruency_congruencyProportion_elecs
unique_congruency_congruencyProportion_elecs = congruency_congruencyProportion_elecs_set - congruency_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruencyEffectAndLWPC_elecs_list = list(overlap_congruencyEffectAndLWPC_elecs)
unique_congruency_elecs_list = list(unique_congruency_elecs)
unique_congruency_congruencyProportion_elecs_list = list(unique_congruency_congruencyProportion_elecs)

# Plotting
# Plot unique electrodes for congruency in one color
fig = plot_on_average(subjects, picks=unique_congruency_elecs_list, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.35, label_every=1)

# Plot unique electrodes for congruency:congruencyProportion in another color
fig = plot_on_average(subjects, picks=unique_congruency_congruencyProportion_elecs_list, rm_wm=False, hemi='both', color=(0.5, 0, 0.5), size=0.35, fig=fig)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_congruencyEffectAndLWPC_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot switch type and switch type:switchProportion

In [28]:
# Convert lists to sets for set operations
switchType_elecs_set = set(switchType_elecs)
switchType_switchProportion_elecs_set = set(switchType_switchProportion_elecs)

# Find overlapping electrodes (present in both lists)
overlap_switchCostAndLWPS_elecs = switchType_elecs_set.intersection(switchType_switchProportion_elecs_set)

# Find unique electrodes in switch type elecs
unique_switchType_elecs = switchType_elecs_set - switchType_switchProportion_elecs_set

# Find unique electrodes in congruency_congruencyProportion_elecs
unique_switchType_switchProportion_elecs = switchType_switchProportion_elecs_set - switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_switchCostAndLWPS_elecs_list = list(overlap_switchCostAndLWPS_elecs)
unique_switchType_elecs_list = list(unique_switchType_elecs)
unique_switchType_switchProportion_elecs_list = list(unique_switchType_switchProportion_elecs)

# Plotting
# Plot unique electrodes for switch type in one color
fig = plot_on_average(subjects, picks=unique_switchType_elecs_list, rm_wm=False, hemi='both', color=(0, 1, 0), size=0.35, label_every=1)

# Plot unique electrodes for switchType:switchProportion in another color
fig = plot_on_average(subjects, picks=unique_switchType_switchProportion_elecs_list, rm_wm=False, hemi='both', color=(0.6, 1, 0.6), size=0.35, fig=fig)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_switchCostAndLWPS_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

plot congruency vs switch type

In [30]:
# Find overlapping electrodes (present in both lists)
overlap_congruencyEffectAndSwitchCost_elecs = switchType_elecs_set.intersection(congruency_elecs_set)

# Find unique electrodes in switchType_elecs
unique_switchType_elecs = switchType_elecs_set - congruency_elecs_set

# Find unique electrodes in congruency_elecs
unique_congruency_elecs = congruency_elecs_set - switchType_elecs_set

# Convert sets back to lists if necessary for plotting functions
overlap_congruencyEffectAndSwitchCost_elecs_list = list(overlap_congruencyEffectAndSwitchCost_elecs)
unique_switchType_elecs_list = list(unique_switchType_elecs)
unique_switchType_switchProportion_elecs_list = list(unique_switchType_switchProportion_elecs)

# Plotting
# Plot unique electrodes for congruency effect
fig = plot_on_average(subjects, picks=unique_congruency_elecs_list, rm_wm=False, hemi='both', color=(1, 0, 0), size=0.35)

# Plot unique electrodes for switchType in another color
fig = plot_on_average(subjects, picks=unique_switchType_elecs_list, rm_wm=False, hemi='both', color=(0, 1, 0), size=0.35, fig=fig)

# Plot overlapping electrodes in black
fig = plot_on_average(subjects, picks=overlap_congruencyEffectAndSwitchCost_elecs_list, rm_wm=False, hemi='both', color=(0, 0, 0), size=0.35, fig=fig)

C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
C:\Users\jz421\Desktop\GlobalLocal\IEEG_Pipelines\ieeg\viz\mri.py:756: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head tra

### plot all stim electrodes for an roi
get the subject: electrode mappings from roi_analysis first

In [ ]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# List of subjects
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

In [ ]:
import json

# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

In [ ]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

this is with zeroes removed from subject names, i think plot on average needs those instead of the normal subject names  
also, remove the subjects with empty lists since those might be breaking the plotting too

In [ ]:
def remove_leading_zeroes_and_empty_subjects(subjects_dict):
    """
    Removes leading zeroes from subject names, adds a '_without_zeroes' suffix, 
    and filters out subjects with empty lists.

    Args:
    subjects_dict (dict): A dictionary with subjects as keys and lists as values.

    Returns:
    dict: A new dictionary with modified subject names and non-empty lists.
    """
    new_dict = {}
    for sub, value in subjects_dict.items():
        if value:  # Check if the list is not empty
            sub_without_zeroes = "D" + sub[1:].lstrip('0')
            new_dict[sub_without_zeroes] = value
    return new_dict

# Example usage:
dlpfc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(dlpfc_electrodes_per_subject)
sig_dlpfc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_dlpfc_electrodes_per_subject)

acc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(acc_electrodes_per_subject)
sig_acc_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_acc_electrodes_per_subject)

parietal_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(parietal_electrodes_per_subject)
sig_parietal_electrodes_without_zeroes = remove_leading_zeroes_and_empty_subjects(sig_parietal_electrodes_per_subject)

### plot sig roi electrodes on individual brains

#### dlpfc

In [ ]:
for subj in sig_dlpfc_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D0103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes])
brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=dlpfc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black')
# brain_fig = plot_subj(subj_without_zeroes, picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black', color=(1, 1, 0))

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_dlPFC.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig dlpfc electrodes

In [ ]:
for subj in sig_dlpfc_electrodes_without_zeroes:
    if subj == 'D57':
        continue #skip this one cuz its being weird
    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_dlpfc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=dlpfc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_dlPFC', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_dlPFC.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

#### acc

In [ ]:
for subj in sig_acc_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D0103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_acc_electrodes_without_zeroes[subj_without_zeroes])

brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_acc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=acc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_acc', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_acc.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig acc electrodes

In [ ]:
for subj in sig_acc_electrodes_without_zeroes:

    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_acc_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_acc_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=acc_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_acc', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_acc.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

#### parietal

In [ ]:
for subj in sig_parietal_electrodes_without_zeroes:
    print(subj)

manually do single subjects

In [ ]:
subj = 'D103'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
print("Picks for subject:", sig_parietal_electrodes_without_zeroes[subj_without_zeroes])

brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_parietal_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=parietal_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_parietal', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_parietal.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

loop thru everyone who has sig parietal electrodes

In [ ]:
for subj in sig_parietal_electrodes_without_zeroes:

    base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

    subj_without_zeroes = "D" + subj[1:].lstrip('0')
    print("Picks for subject:", sig_parietal_electrodes_without_zeroes[subj_without_zeroes])

    brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=sig_parietal_electrodes_without_zeroes[subj_without_zeroes], nonsig_picks=parietal_electrodes_without_zeroes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_parietal', background='black')

    # Specify the path and filename for the PNG file
    png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_parietal.png")

    # Save the figure
    brain_fig.save_image(png_filepath, 'png')

### do dlpfc, acc, parietal coverage for each person

get all electrodes across dlpfc, acc, and parietal and stick them into one dictionary so we can plot coverage in control regions for each subject

In [ ]:
allSigControlElectrodes = {key: sig_parietal_electrodes_without_zeroes.get(key, []) + 
                      sig_dlpfc_electrodes_without_zeroes.get(key, []) + 
                      sig_acc_electrodes_without_zeroes.get(key, [])
                 for key in set(sig_parietal_electrodes_without_zeroes) | 
                             set(sig_dlpfc_electrodes_without_zeroes) | 
                             set(sig_acc_electrodes_without_zeroes)}

allControlElectrodes = {key: parietal_electrodes_without_zeroes.get(key, []) + 
                      dlpfc_electrodes_without_zeroes.get(key, []) + 
                      acc_electrodes_without_zeroes.get(key, [])
                 for key in set(parietal_electrodes_without_zeroes) | 
                             set(dlpfc_electrodes_without_zeroes) | 
                             set(acc_electrodes_without_zeroes)}

subj = 'D63'
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'

subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj_sig_and_nonsig(subj_without_zeroes, sig_picks=allSigControlElectrodes[subj_without_zeroes], nonsig_picks=allControlElectrodes[subj_without_zeroes], sig_color=(1,1,0), nonsig_color=(1,1,1), labels_every=1, hemi='both', title=f'{subj}_allControlElectrodes', background='black')

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_allControlElectrodes.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

### try to plot all sig electrodes in an roi on an avg brain  
this is plotting like all electrodes wtf why

In [ ]:
plot_on_average(sig_parietal_electrodes_without_zeroes, hemi='both', color=(1,1,0))

### plot each subject one by one on the fsaverage brain

In [ ]:
# grab all of the significant electrodes, and put them into one list, and then run plot_on_average

HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)


print(layout.derivatives)
print(layout.derivatives.keys())


base_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")

subjects = layout.get(return_type="id", target="subject")
subjects_without_zeroes = ["D" + subj[1:].lstrip('0') for subj in subjects]
allSigElectrodes = {}  # Initialize as an empty dictionary
allElectrodes = {}

subj = 'D0071'
    
resp_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Response.json'
stim_filename = f'{base_path}\\{subj}\\sig_chans_{subj}_Stimulus.json'

resp_sig_chans = load_sig_chans(resp_filename)
stim_sig_chans = load_sig_chans(stim_filename)

# Combine the significant electrodes for this subject
combined_chans = resp_sig_chans + stim_sig_chans

# Add them to the dictionary, tagged with the subject without zeroes

subj_without_zeroes = "D" + subj[1:].lstrip('0')
allSigElectrodes[subj_without_zeroes] = combined_chans

# Get all electrodes for this subject and add them to the allElectrodes dict
info = subject_to_info(subj_without_zeroes)
allSubjElectrodes = mne.pick_channels(info.ch_names, [])
allElectrodes[subj_without_zeroes] = allSubjElectrodes


plot_on_average(allSigElectrodes, hemi='both', color=(1,1,0))
plot_on_average(allElectrodes, hemi='both', color=(1,1,0))